<a href="https://colab.research.google.com/github/nausheen53/SWE/blob/master/mushroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving test_labels.csv to test_labels.csv
Saving test_set.csv to test_set.csv
Saving train.csv to train.csv
Saving train_set.csv to train_set.csv


In [18]:
import pandas as pd
import csv
import math
from statistics import mode
import numpy
import sklearn
from scipy import stats as s
from sklearn.metrics import *

class KNNClassifier:
    trainingSet=[]
    validationSet=[]
    euclid_dist=[]
    manhattan_dist=[]
    actual_label=[]
    testSet=[]
    train_label=[]
    valid_label=[]
    given_label=[]
    predicted_label=[]
    K_euclid=0
    K_manhat=0
    count=0
    tr=0
    va=0
    
    attr=[["b","c","x","f","k","s"],
          ["f","g","y","s"],
          [" n","b","c","g","r","p","u","e","w","y"],
          ["t","f"],
          ["a","l","c","y","f","m","n","p","s"],
          ["a","d","f","n"],
          ["c","w","d"],
          ["b","n"],
          ["k","n","b","h","g","r","o","p","u","e","w","y"],
          ["e","t"],
          ["f","y","k","s"],
          ["f","y","k","s"],
          ["n","b","c","g","o","p","e","w","y"],
          ["n","b","c","g","o","p","e","w","y"],
          ["p","u"],
          ["n","o","w","y"],
          ["n","o","t"],
          ["c","e","f","l","n","p","s","z"],
          ["k","n","b","h","r","o","u","w","y"],
          ["a","c","n","s","v","y"],
          ["g","l","m","p","u","w","d"]]

    
    
    def calculate_size(self,train):
        c=0
        with open(train) as csvfile:
            readCSV = csv.reader(csvfile)
            for r in readCSV:
                c=c+1
                #print("count",self.count)
                self.count=c
    
    def perform_encoding_training(self,training_csv):
        df = pd.read_csv ('train.csv',header=None)
        #print(df)
        df.drop(df.columns[11],axis=1,inplace=True)
        x = pd.read_csv('train.csv', index_col=False,header=None).iloc[:,0]
        label=[]
        for r in x:
            label.append(r)
        self.actual_label=label
        print(len(self.actual_label),"act label size haii..")
        df.drop(df.columns[0],axis=1,inplace=True)
        df_other=[]
        li=[]
        for j in range(1,23):
            if(j==11):
                #print("ignoring 11")
                continue
            else:
                li=[]
                for i in range(0,len(df)):
                    li.append(df[j][i])
                    # print(li)
                    df_other.append(li)
        print(len(df_other),"len of df other")
        # print(df_other)
        final_df=pd.DataFrame()
        for i in range(0,21):
            # print("performing one hot")
            one_hot= pd.get_dummies(df_other[i],columns= self.attr[i])
            one_hot=one_hot.T.reindex(self.attr[i]).T.fillna(0)
            new_df= pd.DataFrame(one_hot)
            final_df=pd.concat([final_df,new_df],axis=1)
        # print(final_df)
        self.count=len(final_df)
        print(len(final_df),"count")
        self.tr=int(self.count*0.75)
        print(self.tr,"tr")
        self.va=int(self.count*0.25)
        print(self.va,"va")
        # final_df.to_csv(r'/home/nausheen/pythAssing/q2/train_set.csv', index=False,header=0)
        r=0
        for w in range(0,self.tr):
            self.train_label.append(self.actual_label[r])
            r=r+1
        for u in range(0,self.va):
            self.valid_label.append(self.actual_label[r])
            r=r+1


    def encode_test(self,testS):
        df1 = pd.read_csv ('test.csv',header=None)
        df1.drop(df1.columns[10],axis=1,inplace=True)
        df_other1=[]
        li=[]
        for j in range(0,len(df1.columns)+1):
          if(j==10):
            continue
          else:
            li=[]
            for i in range(0,len(df1)):
                li.append(df1[j][i])
            df_other1.append(li)
        final_df1=pd.DataFrame()
        for i in range(0,len(self.attr)):
            one_hot1= pd.get_dummies(df_other1[i],columns=self. attr[i])
            one_hot1=one_hot1.T.reindex(self.attr[i]).T.fillna(0)
            new_df1= pd.DataFrame(one_hot1)
            final_df1=pd.concat([final_df1,new_df1],axis=1)
        # final_df1.to_csv(r'/home/nausheen/pythAssing/q2/test_set.csv', index=False,header=0) 

    
        
    def train(self,training_csv):
        self.perform_encoding_training(training_csv)
#         self.calculate_size(training_csv)
        print(self.count)
#         self.tr=int(self.count*0.8)
        print(self.tr,"train len")
#         self.va=int(self.count*0.2)
        print(self.va,"valid len")
        with open('train_set.csv') as csvfile:
            readCSV = csv.reader(csvfile)
            count=0
            for r in readCSV:
                if count<self.tr:
                    traini=[]
                    for col in r:
                        traini.append(pd.to_numeric(col))
                    self.trainingSet.append(traini)
                    count=count+1
                else:
                    valid=[]
                    for col in r:
                        valid.append(pd.to_numeric(col))
                    self.validationSet.append(valid)
                    count=count+1

        

        
        self.calculate_euclid_dis(self.validationSet,self.trainingSet)
        self.find_K_euclid()
        self.calculate_manhattan_dist(self.validationSet,self.trainingSet)
        self.find_K_manhat()
            
    def do_sort(self,label_dis):
            return(sorted(label_dis, key = lambda x: x[1]))  
        
    def calculate_euclid_dis(self,valSet,trainSet):
            s=[]
            t=[]
            q=0
            print(len(valSet),"vali set len")
            print(len(trainSet),"len of train")
            for i in range(0,len(valSet)):
                s=[]
                s=valSet[i]
                s1=s[1:]
                s1=numpy.array(s1)
                q=q+1
                # print(q,"euc")
                # print("*****************printing s1*************  ")
                #print(s1)
                euclid_dis=[]
                for j in range(0,len(trainSet)):
                    t=[]
                    t=trainSet[j]
                    label=self.train_label[j]
                    t1=t[1:]
                    t1=numpy.array(t1)
                    dis=0
                    label_dis=[]
                    dis = numpy.linalg.norm(s1 - t1)
                    label_dis.append(label)
                    label_dis.append(dis)
                    euclid_dis.append(label_dis)
                euclid_dis_pair=self.do_sort(euclid_dis)
                self.euclid_dist.append(euclid_dis_pair)
            print("euclid dist size in cal euc fun ",len(self.euclid_dist))
            

#                 break

    def calculate_manhattan_dist(self,valSet,trainSet):
                s=[]
                t=[]
                q=0
                for i in range(0,len(valSet)):
                    s=[]
                    s=valSet[i]
                    s1=s[1:]
                    manhattan_dis=[]
                    q=q+1
                    # print(q,"manhat")
                    for j in range(0,len(trainSet)):
                        t=[]
                        t=trainSet[j]
                        label=self.train_label[j]
                        t1=t[1:]
                        m_dis=0
                        label_dis=[]
                        for k in range(0,len(s1)):
                            m_dis=m_dis+ abs(s1[k]-t1[k])
                        label_dis.append(label)
                        label_dis.append(m_dis)
                        manhattan_dis.append(label_dis)
                    manhattan_dis_pair = self.do_sort(manhattan_dis)
                    self.manhattan_dist.append(manhattan_dis_pair)
                    # break
            
    def find_K_euclid(self):
                accu_k=[]
                self.predicted_label=[]
                K=3
                print(len(self.euclid_dist),"len of euc dis")
                while(K<8):
                    print("find k euclid",K)
                    self.predicted_label=[]
                    for i in range (0,len(self.euclid_dist)):
                    # for i in range(0,5):
                        euc=[]
                        euc=self.euclid_dist[i]
                        euc=euc[0:K]
                        #print("***printing euc ***")
                        #print(euc)
                        li=[]
                        for j in range(0,len(euc)):
                            li.append(euc[j][0])
                        try:
                            modi=mode(li)
                        except:
                            modi=li[0]
                        self.predicted_label.append(modi)

                    # actual_label1=actual_label[0:5]
                    accu_k_pair=[]

                    print(len(self.actual_label),"act lab siz")
                    print(len(self.predicted_label),"predicted lab siz")
                    accu_score= sklearn.metrics.accuracy_score(self.valid_label, self.predicted_label, normalize=True, sample_weight=None)
                    print(K)
                    print("valid label ",self.valid_label)
                    print("predicted label ",self.predicted_label)
                    label1=['e','p']
                    print(f1_score(self.valid_label, self.predicted_label, average=None),"f1 score")
                    
                    print(confusion_matrix(self.valid_label, self.predicted_label,labels=label1)," confusion matrix")
                    print("accuracy score for euc ",accu_score)
                    #print(accu_score)
                    accu_k_pair.append(K)
                    accu_k_pair.append(accu_score)
                    accu_k.append(accu_k_pair)
                    K=K+2
                accu_k=self.do_sort(accu_k)
                accu_k.reverse()
                self.K_euclid=accu_k[0][0]
                print("accuracy score final for euc ",accu_k[0][1])
                print(self.K_euclid," K for euclid ")
                
            
    def find_K_manhat(self):
                    self.predicted_label=[]
                    K=3
                    while(K<8):
                        print("find_k_manhat",K)
                        self.predicted_label=[]
                        for i in range(0,len(self.manhattan_dist)):
                            manhat=[]
                            manhat=self.manhattan_dist[i][0:K]
                            li=[]
                            for j in range(0,len(manhat)):
                                li.append(manhat[j][0])
                            try:
                                modi=mode(li)
                            except:
                                modi=li[0]
                            self.predicted_label.append(modi)

                    
                        accu_k_pair=[]
                        accu_k=[]
                        accu_score= sklearn.metrics.accuracy_score(self.valid_label, self.predicted_label, normalize=True, sample_weight=None)
                        f1_score(self.valid_label,self.predicted_label, average=None)
                        print("valid label ",self.valid_label)
                        print("predicted label ",self.predicted_label)
                        print(K)
                        label1=['e','p']
                        print(confusion_matrix(self.valid_label, self.predicted_label,labels=label1)," confusion matrix")
                        print("accuracy score for manhat ",accu_score)
                        print(f1_score(self.valid_label,self.predicted_label, average=None),"f1 score")
                        #print(accu_score)
                        accu_k_pair.append(K)
                        accu_k_pair.append(accu_score)
                        accu_k.append(accu_k_pair)
                        K=K+2
                    accu_k=self.do_sort(accu_k)
                    accu_k.reverse()
                    self.K_manhat=accu_k[0][0]
                    print("accuracy score final for manhat",accu_k[0][1])
                    print(self.K_manhat," K for manhat")
                    
                
    def predict(self,testS):
        self.predicted_label=[]
        self.encode_test(testS)
        print("inside predict")
        with open('test_set.csv') as csvfile:
            readCSV = csv.reader(csvfile)
            for row in readCSV:
                self.testSet.append(pd.to_numeric(row))
        self.euclid_dist=[]       
        self.calculate_euclid_dis(self.testSet,self.trainingSet)
        print(len(self.euclid_dist),"euclid dist list size")
        self.find_k_e()
        return self.predicted_label
    
    def predict_m(self,testS):
        self.predicted_label=[]
        self.manhattan_dist=[]
        self.calculate_manhattan_dist(self.testSet,self.trainingSet)
        print(len(self.manhattan_dist),"manhat dist list size")
        self.find_k_m()
        return self.predicted_label

        
    def find_k_e(self):
        self.predicted_label=[]
        for i in range(0,len(self.euclid_dist)):
            euc=[]
            euc=self.euclid_dist[i][0:self.K_euclid]
            li=[]
            for j in range(0,len(euc)):
                li.append(euc[j][0])
            try:
                modi = mode(li)
            except:
                modi=li[0]
            self.predicted_label.append(modi)
        print("predicted label size ",len(self.predicted_label))
            
        
    def find_k_m(self):
        self.predicted_label=[]
        for i in range(0,len(self.manhattan_dist)):
            man=[]
            man=self.manhattan_dist[i][0:self.K_manhat]
            li=[]
            for j in range(0,len(man)):
                li.append(man[j][0])
            try:
                modi = mode(li)
            except:
                modi=li[0]
            self.predicted_label.append(modi)
        print("predicted label size ",len(self.predicted_label))
            
#         actual_label1=actual_label[0:5]
#         accu_score= sklearn.metrics.accuracy_score(actual_label1, predicted_label, normalize=True, sample_weight=None)
#         print(accu_score)
        
        
        
knn_classifier = KNNClassifier()
knn_classifier.train('train.csv')
# predictions = knn_classifier.predict('test.csv')
# predictions_m=knn_classifier.predict_m('test.csv')
# test_labels = list()
# with open("test_labels.csv") as f:
      # for line in f:
        # test_labels.append((line))
        
# print("accuracy using euclidean distance ")        
# print (accuracy_score(test_labels, predictions))
# print("   accuracy using manhattan distance   ")
# print(accuracy_score(test_labels,predictions_m))


4496 act label size haii..
94416 len of df other
4496 count
3372 tr
1124 va
4496
3372 train len
1124 valid len
1124 vali set len
3372 len of train
euclid dist size in cal euc fun  1124
1124 len of euc dis
find k euclid 3
4496 act lab siz
1124 predicted lab siz
3
valid label  ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'